Introduction

    The point of this project is to answer the stakeholders question of where in Colorado would be the best place for a new Brewery to be built 
    and operated. The stakeholders want to open a new brewery in Colorado but they are not sure of where the best place for the new location to be. 
    I will be looking at brewery density, traffic amounts by area, the types of brewerys in the area to determine the optimal place to build.

Data

    For this project I will be using Foursquare Location Data along side another data set retreived from Kaggle. The data set retreived from Kaggle
    was compiled from beermonthclub.com and was last updated in 2019. The data sets includes the names and addresses for the breweries as well as the
    locations, website, brewery type and more. I plan to use this data combnined with Foursqaure to map out the brewery locations in Colorado 
    and determine where there is a gap in the market that can be filled.

In [1]:
import numpy as np
from geopy.extra.rate_limiter import RateLimiter

Calling the breweires dataset

In [2]:
# The code was removed by Watson Studio for sharing.

,brewery_name,type,address,website,state,state_breweries
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284


CLeaning the data and extracitng what I need

In [3]:
df_colobrew = df_data_1.loc[df_data_1['state'] == 'colorado']
df_colobrew.head()

,brewery_name,type,address,website,state,state_breweries
431,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",...,colorado,182
432,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",...,colorado,182
433,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",...,colorado,182
434,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",...,colorado,182
435,Two Angels Beer Company,ContractBrewery,"8223 S. Quebec, Englewood, Colorado, 80112",...,colorado,182


In [4]:
df_colobrew.reset_index(drop=True)

,brewery_name,type,address,website,state,state_breweries
0,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",...,colorado,182
1,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",...,colorado,182
2,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",...,colorado,182
3,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",...,colorado,182
4,Two Angels Beer Company,ContractBrewery,"8223 S. Quebec, Englewood, Colorado, 80112",...,colorado,182
...,...,...,...,...,...,...
177,Arctic Craft Brewery,Microbrewery-Closed,"2506 Platte Pl, Colorado Springs, Colorado, 80909",http://www.arcticcraftbrewery.com/,colorado,182
178,Anheuser-Busch Inc.- Fort Collins,Microbrewery,"2351 Busch Dr., Fort Collins, Colorado, 80524",http://www.anheuser-busch.com/,colorado,182
179,"Amicas Pizza, Microbrews & More",Brewpub,"136 East Second St., Salida, Colorado, 81201",http://www.amicassalida.com/,colorado,182
180,Alcatraz Brewing Company,Microbrewery,"8405 Park Meadows Center Dr., Littleton, Color...",http://www.calcafe.com/alcatraz/,colorado,182


In [5]:
df_colobrew.head()

,brewery_name,type,address,website,state,state_breweries
431,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",...,colorado,182
432,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",...,colorado,182
433,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",...,colorado,182
434,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",...,colorado,182
435,Two Angels Beer Company,ContractBrewery,"8223 S. Quebec, Englewood, Colorado, 80112",...,colorado,182


In [6]:
df_colobrew.count()

brewery_name       182
type               182
address            182
website            182
state              182
state_breweries    182
dtype: int64

Using Geopy to add Location data to my dataframe

In [9]:
import requests
import json
import time
!pip install geopy

In [10]:
from geopy.geocoders import Nominatim

In [11]:
df_colobrew.reset_index(drop=True, inplace=True)
df_colobrew.head(10)

,brewery_name,type,address,website,state,state_breweries
0,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",...,colorado,182
1,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",...,colorado,182
2,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",...,colorado,182
3,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",...,colorado,182
4,Two Angels Beer Company,ContractBrewery,"8223 S. Quebec, Englewood, Colorado, 80112",...,colorado,182
5,Twisted Pine Brewing Co.,Microbrewery,"3201 Walnut Street, Ste. A, Boulder, Colorado,...",http://www.twistedpinebrewery.com/,colorado,182
6,Trout Creek Brewing Company,Microbrewery,"500 Front St., Fairplay, Colorado, 80440",...,colorado,182
7,Trinidad Brewing Co,Brewpub,"516 Elm St, Trinidad, Colorado, 81082",http://www.trinidadbrewingcompany.com/,colorado,182
8,Tommyknocker Brewery & Pub,Brewpub,"1401 Miner St., Idaho Springs, Colorado, 80452",http://www.tommyknocker.com/,colorado,182
9,Tivoli Brewery,Microbrewery-Closed,"900 Auraria Parkway, Denver, Colorado, 80202",...,colorado,182


In [18]:
df_colobrew = df_colobrew.dropna()

In [20]:
geolocator = Nominatim(timeout=10, user_agent = "myGeo")
location = geolocator.geocode('')
print((location.latitude, location.longitude))


AttributeError: 'NoneType' object has no attribute 'latitude'

In [19]:
df_colobrew['location'] = df_colobrew['address'].apply(geolocator.geocode)
df_colobrew

,brewery_name,type,address,website,state,state_breweries,location,Lat,Long
0,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",...,colorado,182,"(35, East 1st Street, Nederland, Boulder Count...",39.961947,-105.509292
1,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",...,colorado,182,"(1123, Walnut Street, Boulder, Boulder County,...",40.016758,-105.280583
2,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",...,colorado,182,"(100, South Frontage Road West, Vail, Eagle Co...",39.643492,-106.379686
3,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",...,colorado,182,"(8th Avenue, Jackson Field, Greeley, Weld Coun...",40.400010,-104.691147
5,Twisted Pine Brewing Co.,Microbrewery,"3201 Walnut Street, Ste. A, Boulder, Colorado,...",http://www.twistedpinebrewery.com/,colorado,182,"(3201, Walnut Street, Boulder, Boulder County,...",40.020163,-105.251253
...,...,...,...,...,...,...,...,...,...
174,Avery Brewing Company,Microbrewery,"5763 E. Arapahoe Ave #E, Boulder, Colorado, 80303",http://www.averybrewing.com/,colorado,182,"(Arapahoe Avenue, Boulder, Boulder County, Col...",40.014509,-105.243088
175,Atlantis Brewing Company,ContractBrewery,"375 E. 55th Ave., Denver, Colorado, 80237",http://rainbow.rmi.net/~jhicks/ales/,colorado,182,"(East 55th Avenue, Denver, Denver County, Colo...",39.797085,-104.890760
177,Arctic Craft Brewery,Microbrewery-Closed,"2506 Platte Pl, Colorado Springs, Colorado, 80909",http://www.arcticcraftbrewery.com/,colorado,182,"(2506, Platte Place, Knob Hill, Colorado Sprin...",38.836854,-104.781003
178,Anheuser-Busch Inc.- Fort Collins,Microbrewery,"2351 Busch Dr., Fort Collins, Colorado, 80524",http://www.anheuser-busch.com/,colorado,182,"(2351, Busch Drive, Fort Collins, Larimer Coun...",40.618178,-105.001969


I am now creating seperate columns for the Latitude and Longitude values.

In [17]:
df_colobrew['Lat'] = df_colobrew['location'].apply(lambda x: x.latitude if x != None else None)
df_colobrew['Long'] = df_colobrew['location'].apply(lambda x: x.longitude if x != None else None)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [21]:
df_colobrew.head()

,brewery_name,type,address,website,state,state_breweries,location,Lat,Long
0,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",...,colorado,182,"(35, East 1st Street, Nederland, Boulder Count...",39.961947,-105.509292
1,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",...,colorado,182,"(1123, Walnut Street, Boulder, Boulder County,...",40.016758,-105.280583
2,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",...,colorado,182,"(100, South Frontage Road West, Vail, Eagle Co...",39.643492,-106.379686
3,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",...,colorado,182,"(8th Avenue, Jackson Field, Greeley, Weld Coun...",40.400010,-104.691147
5,Twisted Pine Brewing Co.,Microbrewery,"3201 Walnut Street, Ste. A, Boulder, Colorado,...",http://www.twistedpinebrewery.com/,colorado,182,"(3201, Walnut Street, Boulder, Boulder County,...",40.020163,-105.251253


Now I am getting rid of the entries with no location data.

In [22]:
df_colobrew = df_colobrew.dropna()

In [23]:
df_colobrew.head()

,brewery_name,type,address,website,state,state_breweries,location,Lat,Long
0,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",...,colorado,182,"(35, East 1st Street, Nederland, Boulder Count...",39.961947,-105.509292
1,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",...,colorado,182,"(1123, Walnut Street, Boulder, Boulder County,...",40.016758,-105.280583
2,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",...,colorado,182,"(100, South Frontage Road West, Vail, Eagle Co...",39.643492,-106.379686
3,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",...,colorado,182,"(8th Avenue, Jackson Field, Greeley, Weld Coun...",40.400010,-104.691147
5,Twisted Pine Brewing Co.,Microbrewery,"3201 Walnut Street, Ste. A, Boulder, Colorado,...",http://www.twistedpinebrewery.com/,colorado,182,"(3201, Walnut Street, Boulder, Boulder County,...",40.020163,-105.251253


In [24]:
df_colobrew.reset_index(drop=True, inplace=True)
df_colobrew.head()

,brewery_name,type,address,website,state,state_breweries,location,Lat,Long
0,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",...,colorado,182,"(35, East 1st Street, Nederland, Boulder Count...",39.961947,-105.509292
1,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",...,colorado,182,"(1123, Walnut Street, Boulder, Boulder County,...",40.016758,-105.280583
2,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",...,colorado,182,"(100, South Frontage Road West, Vail, Eagle Co...",39.643492,-106.379686
3,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",...,colorado,182,"(8th Avenue, Jackson Field, Greeley, Weld Coun...",40.400010,-104.691147
4,Twisted Pine Brewing Co.,Microbrewery,"3201 Walnut Street, Ste. A, Boulder, Colorado,...",http://www.twistedpinebrewery.com/,colorado,182,"(3201, Walnut Street, Boulder, Boulder County,...",40.020163,-105.251253


In [25]:
df_colobrew = df_colobrew.drop(['website','state_breweries'], axis=1)
df_colobrew.head()

,brewery_name,type,address,state,location,Lat,Long
0,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",colorado,"(35, East 1st Street, Nederland, Boulder Count...",39.961947,-105.509292
1,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",colorado,"(1123, Walnut Street, Boulder, Boulder County,...",40.016758,-105.280583
2,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",colorado,"(100, South Frontage Road West, Vail, Eagle Co...",39.643492,-106.379686
3,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",colorado,"(8th Avenue, Jackson Field, Greeley, Weld Coun...",40.400010,-104.691147
4,Twisted Pine Brewing Co.,Microbrewery,"3201 Walnut Street, Ste. A, Boulder, Colorado,...",colorado,"(3201, Walnut Street, Boulder, Boulder County,...",40.020163,-105.251253


In [26]:
df_colobrew.shape

(124, 7)

Now I am using the location data to make a map off all brewery locations in colorado.

In [27]:
!pip install folium

     |████████████████████████████████| 94 kB 7.4 MB/s  eta 0:00:01


In [28]:

import folium

map_colobrew = folium.Map(location=[39.5501, -105.7821], zoom_start=7.4)


for lat, lng in zip(df_colobrew['Lat'], df_colobrew['Long']):
    label = '{}'.format('type')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_colobrew)  
    
map_colobrew

Now I am going to conncet to the Foursquare API

In [29]:

CLIENT_ID = 'NBZTJEQLZNCUJ3WRMU1WRRXKNIEVENNHAW4ELGLR1NTRJXTS' # your Foursquare ID
CLIENT_SECRET = 'SENZSW1UA5JW3SPKI5QA55QCOFPOHWIPVAVW1KYWARDUXZYT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NBZTJEQLZNCUJ3WRMU1WRRXKNIEVENNHAW4ELGLR1NTRJXTS
CLIENT_SECRET:SENZSW1UA5JW3SPKI5QA55QCOFPOHWIPVAVW1KYWARDUXZYT


In [30]:
import requests 
from pandas.io.json import json_normalize

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['brewery_name', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
colo_venues = getNearbyVenues(names=df_colobrew['brewery_name'],
                                   latitudes=df_colobrew['Lat'],
                                   longitudes=df_colobrew['Long'])

In [33]:
colo_venues.head()

,brewery_name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wolf Tongue Brewery,39.961947,-105.509292,Crosscut Pizzeria and Taphouse,39.961745,-105.510257,Pizza Place
1,Wolf Tongue Brewery,39.961947,-105.509292,New Moon Bakery and Cafe,39.962122,-105.510616,Bakery
2,Wolf Tongue Brewery,39.961947,-105.509292,The Very Nice Brewing Company,39.959972,-105.509239,Brewery
3,Wolf Tongue Brewery,39.961947,-105.509292,Kathmandu,39.962134,-105.511457,Indian Restaurant
4,Wolf Tongue Brewery,39.961947,-105.509292,Salto Coffee Works,39.962398,-105.508568,Coffee Shop


In [34]:
colo_venues.groupby('brewery_name').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
brewery_name,,,,,,
Adolph Coors Co.,54,54,54,54,54,54
Anheuser-Busch Inc.- Fort Collins,1,1,1,1,1,1
Arctic Craft Brewery,10,10,10,10,10,10
Atlantis Brewing Company,1,1,1,1,1,1
Avery Brewing Company,8,8,8,8,8,8
...,...,...,...,...,...,...
Twisted Pine Brewing Co.,48,48,48,48,48,48
Union Colony Brewery,15,15,15,15,15,15
Vail Brewing Company,29,29,29,29,29,29


Onehot code to prepare data for K-Means

In [35]:

colo_onehot = pd.get_dummies(colo_venues[['Venue Category']], prefix="", prefix_sep="")


colo_onehot['brewery_name'] = colo_venues['brewery_name'] 


fixed_columns = [colo_onehot.columns[-1]] + list(colo_onehot.columns[:-1])
colo_onehot = colo_onehot[fixed_columns]

colo_onehot.head()

,brewery_name,ATM,Accessories Store,Alternative Healer,American Restaurant,Apres Ski Bar,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Wolf Tongue Brewery,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Wolf Tongue Brewery,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Wolf Tongue Brewery,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Wolf Tongue Brewery,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Wolf Tongue Brewery,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
colo_grouped = colo_onehot.groupby('brewery_name').mean().reset_index()
colo_grouped

,brewery_name,ATM,Accessories Store,Alternative Healer,American Restaurant,Apres Ski Bar,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adolph Coors Co.,0.000000,0.0,0.0,0.018519,0.0,0.0,0.0,0.018519,0.0,...,0.0,0.0,0.0,0.0,0.0,0.018519,0.0,0.0,0.000000,0.000000
1,Anheuser-Busch Inc.- Fort Collins,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,Arctic Craft Brewery,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,Atlantis Brewing Company,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Avery Brewing Company,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Twisted Pine Brewing Co.,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.041667,0.020833
117,Union Colony Brewery,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
118,Vail Brewing Company,0.000000,0.0,0.0,0.068966,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
119,Walnut Brewery/Rock Bottom Restaurant,0.012658,0.0,0.0,0.025316,0.0,0.0,0.0,0.012658,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [37]:
from sklearn.cluster import KMeans

kclusters = 5

colo_grouped_clustering = colo_grouped.drop('brewery_name', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(colo_grouped_clustering)

kmeans.labels_[0:10]

array([0, 2, 0, 1, 4, 0, 0, 0, 0, 0], dtype=int32)

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['brewery_name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['brewery_name'] = colo_grouped['brewery_name']

for ind in np.arange(colo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(colo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,brewery_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adolph Coors Co.,Brewery,Bar,Coffee Shop,Sandwich Place,Pizza Place,Hotel,Gym,Café,Pet Store,College Library
1,Anheuser-Busch Inc.- Fort Collins,Restaurant,Yoga Studio,Fish Market,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
2,Arctic Craft Brewery,Diner,Vietnamese Restaurant,Marijuana Dispensary,Sandwich Place,Discount Store,Bank,Restaurant,Donut Shop,Furniture / Home Store,Flea Market
3,Atlantis Brewing Company,Park,Yoga Studio,Escape Room,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
4,Avery Brewing Company,Thrift / Vintage Store,Auto Dealership,Board Shop,Café,Bank,Lawyer,Brewery,Pizza Place,Diner,Fondue Restaurant


In [41]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

colo_merged = df_colobrew

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
colo_merged = colo_merged.join(neighborhoods_venues_sorted.set_index('brewery_name'), on='brewery_name')

colo_merged.head() # check the last columns!

,brewery_name,type,address,state,location,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wolf Tongue Brewery,Microbrewery-Closed,"35 E. 1st St., Nederland, Colorado, 80466",colorado,"(35, East 1st Street, Nederland, Boulder Count...",39.961947,-105.509292,0.0,Coffee Shop,Ice Cream Shop,Pizza Place,Indian Restaurant,Jewelry Store,Supermarket,Café,Shop & Service,Thai Restaurant,Bakery
1,Walnut Brewery/Rock Bottom Restaurant,Brewpub,"1123 Walnut St., Boulder, Colorado, 80302",colorado,"(1123, Walnut Street, Boulder, Boulder County,...",40.016758,-105.280583,0.0,New American Restaurant,Arts & Crafts Store,Sushi Restaurant,Ice Cream Shop,Coffee Shop,Taco Place,Sporting Goods Shop,Italian Restaurant,Hotel,Hobby Shop
2,Vail Brewing Company,ContractBrewery,"100 S. Frontage Rd. W, Vail, Colorado, 81657",colorado,"(100, South Frontage Road West, Vail, Eagle Co...",39.643492,-106.379686,0.0,Hotel,Spa,American Restaurant,French Restaurant,Lounge,Pizza Place,North Indian Restaurant,Brewery,Skating Rink,Sports Bar
3,Union Colony Brewery,Brewpub,"1412 8th Ave., Greeley, Colorado, 80631",colorado,"(8th Avenue, Jackson Field, Greeley, Weld Coun...",40.400010,-104.691147,0.0,Mexican Restaurant,Fast Food Restaurant,Pizza Place,Bowling Alley,Veterans' Organization,Liquor Store,Chinese Restaurant,Grocery Store,Sandwich Place,Seafood Restaurant
4,Twisted Pine Brewing Co.,Microbrewery,"3201 Walnut Street, Ste. A, Boulder, Colorado,...",colorado,"(3201, Walnut Street, Boulder, Boulder County,...",40.020163,-105.251253,0.0,Coffee Shop,Gym,Brewery,Supplement Shop,Lingerie Store,Clothing Store,Women's Store,Sporting Goods Shop,Café,Martial Arts School


In [42]:
colobrew_map = colo_merged.dropna()

In [43]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [44]:
# create map
map_clusters = folium.Map(location=[39.5501, -105.7821], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(colobrew_map['Lat'], colobrew_map['Long'], colobrew_map['brewery_name'], colobrew_map['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

From What we can see there are more breweires in cluster on than the other clusters by far.

In [45]:
colobrew_map.loc[colobrew_map['Cluster Labels'] == 0, colobrew_map.columns[[1] + list(range(5, colobrew_map.shape[1]))]]

,type,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Microbrewery-Closed,39.961947,-105.509292,0.0,Coffee Shop,Ice Cream Shop,Pizza Place,Indian Restaurant,Jewelry Store,Supermarket,Café,Shop & Service,Thai Restaurant,Bakery
1,Brewpub,40.016758,-105.280583,0.0,New American Restaurant,Arts & Crafts Store,Sushi Restaurant,Ice Cream Shop,Coffee Shop,Taco Place,Sporting Goods Shop,Italian Restaurant,Hotel,Hobby Shop
2,ContractBrewery,39.643492,-106.379686,0.0,Hotel,Spa,American Restaurant,French Restaurant,Lounge,Pizza Place,North Indian Restaurant,Brewery,Skating Rink,Sports Bar
3,Brewpub,40.400010,-104.691147,0.0,Mexican Restaurant,Fast Food Restaurant,Pizza Place,Bowling Alley,Veterans' Organization,Liquor Store,Chinese Restaurant,Grocery Store,Sandwich Place,Seafood Restaurant
4,Microbrewery,40.020163,-105.251253,0.0,Coffee Shop,Gym,Brewery,Supplement Shop,Lingerie Store,Clothing Store,Women's Store,Sporting Goods Shop,Café,Martial Arts School
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Brewpub-Closed,37.936921,-107.812255,0.0,American Restaurant,Hotel,Bar,Art Gallery,Clothing Store,Mexican Restaurant,Pizza Place,Sandwich Place,Café,Ski Area
117,Microbrewery,38.878642,-104.815977,0.0,Bakery,Fast Food Restaurant,Pet Store,Storage Facility,Gas Station,Park,Coffee Shop,Brewery,Home Service,American Restaurant
118,Microbrewery,38.833730,-104.826654,0.0,Coffee Shop,Mexican Restaurant,Sandwich Place,Bar,Hotel,Art Gallery,French Restaurant,Sushi Restaurant,Mobile Phone Shop,Italian Restaurant
121,Microbrewery-Closed,38.836854,-104.781003,0.0,Diner,Vietnamese Restaurant,Marijuana Dispensary,Sandwich Place,Discount Store,Bank,Restaurant,Donut Shop,Furniture / Home Store,Flea Market


In [46]:
colobrew_map.loc[colobrew_map['Cluster Labels'] == 1, colobrew_map.columns[[1] + list(range(5, colobrew_map.shape[1]))]]

,type,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Microbrewery,39.797085,-104.89076,1.0,Park,Yoga Studio,Escape Room,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
120,ContractBrewery,39.797085,-104.89076,1.0,Park,Yoga Studio,Escape Room,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market


In [47]:
colobrew_map.loc[colobrew_map['Cluster Labels'] == 2, colobrew_map.columns[[1] + list(range(5, colobrew_map.shape[1]))]]

,type,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
122,Microbrewery,40.618178,-105.001969,2.0,Restaurant,Yoga Studio,Fish Market,Event Space,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market


In [48]:
colobrew_map.loc[colobrew_map['Cluster Labels'] == 3, colobrew_map.columns[[1] + list(range(5, colobrew_map.shape[1]))]]

,type,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,Microbrewery,38.795078,-104.776752,3.0,Convenience Store,Business Service,Yoga Studio,Fish Market,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market


In [49]:
colobrew_map.loc[colobrew_map['Cluster Labels'] == 4, colobrew_map.columns[[1] + list(range(5, colobrew_map.shape[1]))]]

,type,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Microbrewery,40.589524,-105.063333,4.0,Brewery,Food Truck,Gym,Hobby Shop,Breakfast Spot,Deli / Bodega,Department Store,Cupcake Shop,Food Service,Food Court
50,Microbrewery,40.593358,-105.067842,4.0,Brewery,Racetrack,Performing Arts Venue,Theater,Gift Shop,Park,Event Space,Exhibit,Fabric Shop,Farmers Market
59,Microbrewery,40.158839,-105.112895,4.0,Brewery,Beer Garden,Lounge,Yoga Studio,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market
60,Brewpub,39.085347,-108.551935,4.0,Coffee Shop,Hardware Store,Brewery,Pizza Place,Pharmacy,Burger Joint,Sandwich Place,Event Space,Exhibit,Fabric Shop
82,Microbrewery,40.023011,-105.238350,4.0,Rental Car Location,Breakfast Spot,Brewery,Food Truck,American Restaurant,Dance Studio,Gym / Fitness Center,Fish Market,Fabric Shop,Farmers Market
89,Brewpub,37.473230,-108.504562,4.0,Coffee Shop,Brewery,American Restaurant,Gas Station,Burger Joint,Yoga Studio,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
93,Microbrewery,40.431405,-104.688593,4.0,Brewery,Food Truck,Discount Store,Construction & Landscaping,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
110,Brewpub,39.159206,-108.732698,4.0,Brewery,Café,Pizza Place,Mexican Restaurant,Grocery Store,Gym / Fitness Center,Pharmacy,Coffee Shop,Exhibit,Fabric Shop
119,Microbrewery,40.014509,-105.243088,4.0,Thrift / Vintage Store,Auto Dealership,Board Shop,Café,Bank,Lawyer,Brewery,Pizza Place,Diner,Fondue Restaurant


In [50]:
colobrew_map.loc[colobrew_map['Cluster Labels'] == 5, colobrew_map.columns[[1] + list(range(5, colobrew_map.shape[1]))]]

,type,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
